In [78]:
import torch
import torchaudio.datasets as datasets

In [6]:
dataset = datasets.LJSPEECH("../data")

In [3]:
from torch.utils.data import DataLoader

In [11]:
dataloader = DataLoader(dataset, batch_size=64)

In [95]:
from torch.nn.functional import pad
from torchaudio.functional import resample

def collate_fn(batched_data, resample_rate=0):
    if resample_rate > 0:
        resampled_waveforms = list(map(lambda x: resample(x[0], x[1], resample_rate), batched_data))
        batched_data = [(resampled_waveforms[i], resample_rate, batched_data[i][2], batched_data[i][3]) 
                        for i in range(len(batched_data))]
    
    max_size = max(list(map(lambda x: x[0].size(-1), batched_data)))
    padded_waveforms = list(map(lambda x: pad(x[0], (0, max_size-x[0].size(-1))), batched_data))
    stacked_waveforms = torch.stack(padded_waveforms)
    
    normalized_transcripts = list(map(lambda x: x[3], batched_data))

    return (stacked_waveforms, normalized_transcripts)

In [96]:
dataloader = DataLoader(dataset, batch_size=64, collate_fn=collate_fn_16000)

In [97]:
for item in dataloader: break